In [ ]:
import random
import numpy as np
import matplotlib as plt
import json
import glob
import torch
import re
import copy

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/cs6471proj2/amazon

Mounted at /content/drive
/content/drive/MyDrive/cs6471proj2/amazon


# Combine the annotation results from two annotators

In [ ]:
with open('purpose_classification/complete_idx_label_ruiqi2.json', 'r') as f:
    data1 = json.load(f)

with open('purpose_classification/complete_idx_label_zexing4.json', 'r') as f:
    data2 = json.load(f)

data = set()
for k, item in data1.items():
    data.add((k, item[0]))

for k, item in data2.items():
    data.add((k, item[0]))

assert len(data) ==  (len(data1) + len(data2))

print('DATA AMOUNT:', len(data))

data = list(data)

random.shuffle(data)

eval = data[-400:]
training = data[:-400]

with open('purpose_classification_new/training_idx_target.json', 'w') as f:
    json.dump(training, f)

with open('purpose_classification_new/eval_idx_target.json', 'w') as f:
    json.dump(eval, f)

DATA AMOUNT: 3737


# Anotation data anslysis

In [ ]:
country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 
                      'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'],  'australia':['australia'], 
                      'italy':['italy'], 'france':['france'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'brazil':['brazil'], 'korea':['korea'], 
                      'afghanistan':['afghanistan'], 'spain':['spain'], 'new zealand':['new zealand'], 
                      'singapore':['singapore'], 'iraq':['iraq'], 'russia':['russia'], 'switzerland':['switzerland']}


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

path2newfiles = 'purpose_classification'
file_handle = open('/'.join([path2newfiles, 'global_country_idx2data.json']), 'r')
idx2data = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'common_country_sample_idx.json']), 'r')
common_country_sample_idx = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_token.json']), 'r')
token = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_data.json']), 'r')
pos = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country2idx.json']), 'r')
global_country2idx = json.load(file_handle)
file_handle.close()

In [ ]:

with open('purpose_classification/complete_idx_label_ruiqi2.json', 'r') as f:
    zexing = json.load(f)
with open('purpose_classification/complete_idx_label_zexing4.json', 'r') as f:
    ruiqi = json.load(f)

# idx2country:
idx2country = {}
for k, item in global_country2idx.items():
    for i in item:
        idx2country[int(i)] = k
    
print(len(zexing)+len(ruiqi))
total = copy.deepcopy(zexing)
for k, item in ruiqi.items():
    total[k] = item

print(len(total))

dict_ = {}
for k, item in total.items():
    if idx2country[int(k)] not in word2label_country:
        print(idx2country[int(k)])
        continue
    country = word2label_country[idx2country[int(k)]]
    if country not in dict_:
        dict_[country] = {item[0]:1}
    else:
        if item[0] not in dict_[country]:
            dict_[country][item[0]] = 1
        else:
            dict_[country][item[0]] += 1

print(dict_)

3737
3737
{'us': {2: 388, 1: 705, 3: 209, 4: 326, 5: 77}, 'china': {1: 829, 5: 35, 4: 11, 2: 13, 3: 10}, 'uk': {3: 89, 2: 84, 1: 70, 4: 109, 5: 13}, 'russia': {5: 1, 1: 3, 4: 3, 2: 2, 3: 4}, 'brazil': {1: 5, 2: 1, 4: 8, 3: 1}, 'afghanistan': {3: 9, 2: 1, 4: 5}, 'mexico': {2: 13, 1: 39, 3: 18, 4: 14}, 'vietnam': {3: 7, 2: 10, 1: 10, 4: 11}, 'canada': {2: 11, 3: 20, 1: 22, 4: 7}, 'thailand': {1: 11, 2: 3, 4: 1, 3: 8}, 'iraq': {3: 3, 2: 4, 4: 5, 1: 1}, 'japan': {1: 44, 3: 23, 2: 13, 4: 13}, 'germany': {1: 46, 2: 8, 3: 13, 4: 5}, 'italy': {1: 14, 4: 11, 3: 16, 2: 6}, 'australia': {3: 24, 1: 12, 2: 13, 4: 10}, 'france': {3: 12, 2: 5, 1: 12, 4: 12}, 'india': {4: 23, 1: 26, 3: 14, 2: 7, 5: 2}, 'taiwan': {1: 29, 2: 1, 4: 1}, 'switzerland': {2: 2, 1: 11, 3: 3, 4: 1}, 'korea': {1: 14, 2: 1, 4: 5, 3: 1}, 'singapore': {1: 6, 3: 6, 4: 6, 2: 6}, 'spain': {3: 2, 2: 4, 4: 9, 1: 3}, 'new zealand': {3: 5, 4: 4, 1: 3, 2: 1}}


In [ ]:
stat = {1:0, 2:0, 3:0, 4:0, 5:0}
for k, item in zexing.items():
    stat[int(item[0])] += 1

print(stat)

stat1 = {1:0, 2:0, 3:0, 4:0, 5:0}
for k, item in ruiqi.items():
    stat1[int(item[0])] += 1

print(stat1)

for k, item in stat.items():
    stat1[k] += item

print(stat1)


{1: 592, 2: 220, 3: 145, 4: 160, 5: 31}
{1: 1323, 2: 377, 3: 352, 4: 440, 5: 97}
{1: 1915, 2: 597, 3: 497, 4: 600, 5: 128}


In [ ]:
# {1: 592, 2: 220, 3: 145, 4: 160, 5: 31}
# {1: 788, 2: 248, 3: 241, 4: 167, 5: 66}
# {1: 1380, 2: 468, 3: 386, 4: 327, 5: 97}
1380/(1380+468+386+327+97)

0.5191873589164786

In [ ]:
# {1: 592, 2: 220, 3: 145, 4: 160, 5: 31}
# {1: 1323, 2: 377, 3: 352, 4: 440, 5: 97}
# {1: 1915, 2: 597, 3: 497, 4: 600, 5: 128}

# BERT RELATED CODE

## model

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 51.8MB/s 
     |████████████████████████████████| 901kB 55.6MB/s 


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import json

class HDCTModel(BertModel):
    """
    A model wrapper around hf BERT implementation
    """
    def __init__(self, config):
        super().__init__(config=config)
        self.fc = nn.Linear(config.hidden_size, 5)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, country_mask, input_ids, token_type_ids, attention_mask):
        outputs = super().forward(input_ids, attention_mask, token_type_ids)
        last_hidden = outputs[0]    # (batch_size, seq_len, hidden_size)
        country_hidden_total = torch.sum(last_hidden*country_mask.unsqueeze(-1), dim=1).squeeze()# (batch_size, hidden_size)
        country_hidden_n = torch.sum(country_mask, dim=-1).view(last_hidden.size(0), 1)
        country_hidden = country_hidden_total/country_hidden_n
        # import pdb
        # pdb.set_trace()
        logits = self.softmax(self.fc(country_hidden))    # (batch_size, 5)
        # print(logits.size())

        return logits

## dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class Country_purpose_dataset(Dataset):
    def __init__(self, training=True, entire=False):

        self.preprocess_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        if training:
            if entire:
                with open('purpose_classification_new/training_idx_target.json', 'r') as f:
                    self.idx_target = json.load(f)
                with open('purpose_classification_new/eval_idx_target.json', 'r') as f:
                    mode_idx_target = json.load(f) 
                for idx, t in mode_idx_target:
                    self.idx_target.append((idx, t))
            else:
                with open('purpose_classification_new/training_idx_target.json', 'r') as f:
                    self.idx_target = json.load(f)
        else:
            if entire:
                pass 
            else:
                with open('purpose_classification_new/eval_idx_target.json', 'r') as f:
                    self.idx_target = json.load(f)


        
        print('DATA AMOUNT:', len(self.idx_target))
            
        # self.idx_target = list(self.data) # data_idx, target

        path2newfiles = 'purpose_classification'
        file_handle = open('/'.join([path2newfiles, 'global_country_idx2data.json']), 'r')
        self.idx2data = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'common_country_sample_idx.json']), 'r')
        self.common_country_sample_idx = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country_token.json']), 'r')
        self.token = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country_data.json']), 'r')
        self.pos = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country2idx.json']), 'r')
        self.global_country2idx = json.load(file_handle)
        file_handle.close()

        # get full list of country for each cat_name+row_idx
        done = ["reviews_Musical_Instruments_5",  "reviews_Automotive_5",
                "reviews_Cell_Phones_and_Accessories_5", "reviews_Toys_and_Games_5", 
                "reviews_Office_Products_5", "reviews_Pet_Supplies_5",
                "reviews_Digital_Music_5", 'reviews_Tools_and_Home_Improvement_5',
                "reviews_Clothing_Shoes_and_Jewelry_5", "reviews_Amazon_Instant_Video_5"]

        find_complete_country_list = {}
        incomplete_data = []
        for index, target in self.idx_target:
            info = self.idx2data[index]
            text = self.token[str(info['category'])][str(info['row_idx'])][1]
            incomplete_data.append((index, target, text, info))
            find_complete_country_list[(info['category'], int(info['row_idx']))] = 0
        
        for name in done:
            file_path = '/'.join([name+'_all', name])
            for row_idx, data in enumerate(open(file_path+'_feature.json', 'r')):
                if (name, row_idx) in find_complete_country_list:
                    feature = json.loads(data)
                    assert len(feature['COUNTRY']) > 0
                    find_complete_country_list[(name, row_idx)] = feature['COUNTRY']
        
        # combine all data to self.data
        raw_data = []
        skip_list = []
        skip_c = 0
        for data in incomplete_data:
            index, target, text, info = data
            assert len(find_complete_country_list[(info['category'], int(info['row_idx']))]) != 0
            country_list = find_complete_country_list[(info['category'], info['row_idx'])]

            # if  self.pos[str(index)]['end'] >= 400 or int(info['country_idx']) >= 6 or index in ['22154', '26418', '29455', '7214', '500', '7284', '24013', '6981', '2850']:
            if self.pos[str(index)]['end'] >= 400 or index in ['22154', '26418', '29455', '7214', '500', '7284', '24013', '6981', '2850']:
                skip_list.append(index)
                skip_c += 1
                continue
            raw_data.append((index, target, text, info, country_list, self.pos[str(index)]['start'], self.pos[str(index)]['end'])) # data_idx, target, text, info(info['country_idx'], info['country_name']), country_list, start, end
        print(skip_c)

        self.data = []
        for r_d in raw_data:
            if self.check_if_tokenize_can_find_country(r_d, r_d[2]):
                self.data.append(r_d)
            else:
                skip_c += 1
                skip_list.append(r_d[0])
        
        print('SKIP NUMBER', skip_c)
        print('SKIP LIST', skip_list)

        if training:
            if entire:
                with open('purpose_classification_new/entire_training_complete_data.json', 'w') as f:
                    json.dump(self.data, f)
                with open('purpose_classification_new/entire_training_skipped_data.json', 'w') as f:
                    json.dump(skip_list, f)
            else:
                with open('purpose_classification_new/training_complete_data.json', 'w') as f:
                    json.dump(self.data, f)
                with open('purpose_classification_new/training_skipped_data.json', 'w') as f:
                    json.dump(skip_list, f)
        else:
            if entire:
                with open('purpose_classification_new/inference_complete_data.json', 'w') as f:
                    json.dump(self.data, f)
                with open('purpose_classification_new/inference_skipped_data.json', 'w') as f:
                    json.dump(skip_list, f)
            else:
                with open('purpose_classification_new/eval_complete_data.json', 'w') as f:
                    json.dump(self.data, f)
                with open('purpose_classification_new/eval_skipped_data.json', 'w') as f:
                    json.dump(skip_list, f)
    
    def check_if_tokenize_can_find_country(self, r_d, text):
            d = {}
            d['country_name'] = r_d[3]['country_name']
            d['country_idx'] = r_d[3]['country_idx']
            d['country_list'] = r_d[-3]
            d['start'] = r_d[-2]
            d['end'] = r_d[-1]
            tokens = self.preprocess_tokenizer(text.lower(), return_tensors="pt", padding='max_length', max_length=512, truncation=True)
            input_ids, _, _ = tokens['input_ids'], tokens['token_type_ids'], tokens['attention_mask']
            start, end = find_country_idx(d, self.preprocess_tokenizer.convert_ids_to_tokens(input_ids[0]))

            if start == 1000 and end == 1000:
                return False
            else:
                return True

        
    def __getitem__(self, idx):
        # (text, target, country_name, country_idx, country_list)
        return {'index':self.data[idx][0], 'text':self.data[idx][2], 'target':int(self.data[idx][1])-1, 'country_name':self.data[idx][3]['country_name'], 
                'country_idx':self.data[idx][3]['country_idx'], 'country_list':self.data[idx][-3], 'start':self.data[idx][-2], 'end':self.data[idx][-1]}
    
    def __len__(self):
        return len(self.data)

def find_country_idx(d, tokenized_sent):
    country_name, country_idx, country_list, start, end = d['country_name'], d['country_idx'], d['country_list'], d['start'], d['end']
    words_length = len(country_name.split(' '))
    record = []
    new_sent = []
    # print(tokenized_sent)
    for idx, t in enumerate(tokenized_sent):
        if idx == 0:
            last_t = t
            record.append(1)
            new_sent.append(t)
            continue
        
        if t.startswith('##'):
            record[-1] += 1
            new_sent[-1] = new_sent[-1] + t[2:]
        else:
            record.append(1)
            new_sent.append(t)

    assert len(record) == len(new_sent)
    # print(record)
    # print(new_sent)
    pointer = 0
    start = 0
    loop = 0
    while 1:
        cur_words = country_list[pointer].lower().split(' ')
        cur_words_len = len(cur_words)
        # print('WORD TO FIND:', cur_words)
        for i in range(start, len(new_sent)-cur_words_len, 1):

            if len(re.sub("[^A-Za-z']+", '', new_sent[i])) == 0:
                continue

            w2compare = [re.sub("[^A-Za-z']+", '', new_sent[i])]
        
            j = i+1
            while len(w2compare) < cur_words_len:
                if len(re.sub("[^A-Za-z']+", '', new_sent[j])) > 0:
                    w2compare.append(new_sent[j])
                j += 1
            
            if w2compare == cur_words:
                if country_idx != pointer:
                    # print('CURRENT WORD==WORD TO FIND', new_sent[i:j], cur_words, country_idx, pointer, i)
                    pointer += 1
                    start = i+cur_words_len
                    break
                else:
                    # print('CURRENT WORD==WORD TO FIND and RETURN', new_sent[i:j], cur_words, country_idx, pointer, i)
                    start_ = sum(record[:i])
                    end_ = sum(record[:j])
                    # print(tokenized_sent[start_:end_], new_sent[i:j])
                    return start_, end_

        if i == len(new_sent)-cur_words_len-1:
            print('SKIP')
            return 1000, 1000

    if start_ == 0 and end_ == 0:
        print('ERROR')
        import pdb
        pdb.set_trace()
    
    return start_, end_

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
def collate_fn(data):
    # tokenized sentence batchxsequence x3
    # target batchx1
    # country_mask batchxsequence
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    targets = []
    country_mask = torch.zeros((len(data), 512))
    for idx,d in enumerate(data):
        # print(d['target'])
        tokens = tokenizer(d['text'].lower(), return_tensors="pt", padding='max_length', max_length=512, truncation=True)
        input_ids, token_type_ids, attention_mask = tokens['input_ids'], tokens['token_type_ids'], tokens['attention_mask']
        start, end = find_country_idx(d, tokenizer.convert_ids_to_tokens(input_ids[0]))
        if start == 1000 and end == 1000:
            print('ERROR')
            import pdb
            pdb.set_trace()
        country_mask[idx, start:end] = 1
        targets.append(int(d['target']))
        # import pdb
        # pdb.set_trace()
        input_ids_list.append(input_ids)
        token_type_ids_list.append(token_type_ids)
        attention_mask_list.append(attention_mask)
    # import pdb
    # pdb.set_trace()
    targets = torch.LongTensor(targets)
    input_ids_list = torch.cat(input_ids_list, dim=0)
    token_type_ids_list = torch.cat(token_type_ids_list, dim=0)
    attention_mask_list = torch.cat(attention_mask_list, dim=0)
    
    return country_mask, input_ids_list, token_type_ids_list, attention_mask_list, targets
    


## training

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 122kB 13.4MB/s 


In [ ]:
lr = 0.000001
# eps = ? 
# weight_decay = ? 
epochs = 50
eval_steps = 1
output_dir = 'purpose_classification_new/training' # output directory to save the best models 

In [ ]:
# train
import logging 
import torch.optim as optim
import os
from tensorboardX import SummaryWriter

class Trainer: 
    def __init__(self, model, device, train_dataloader, dev_dataloader):
        self.model = model
        self.device = device
        self.train_dataloader = train_dataloader
        self.dev_dataloader = dev_dataloader
        self.criterion = nn.NLLLoss(reduction="mean")
        self.training_writter = SummaryWriter(output_dir+'/tensorboard')
        self.testing_writter = SummaryWriter(output_dir+'/tensorboard')
        self.global_step = 0

        

    def train(self):
        self.model.train()
        # optimizer = Adamw(params=self.model.parameters(), lr=lr, eps=eps, weight_decay=weight_decay)
        optimizer = optim.AdamW(params=self.model.parameters(), lr=lr) 
        global_step = 0
        best_eval_loss = None
        train_loss = 0
        recall = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
        prec = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
        for epoch in range(epochs):
            print(epoch)

            if epoch % eval_steps == 0:
                self.model.eval()

                eval_loss = 0
                eval_recall = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
                eval_prec = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
                for step, example in enumerate(self.dev_dataloader):
                    # get outputs
                    country_mask, input_ids, token_type_ids, attention_mask, labels = example
                    country_mask = country_mask.to(self.device)
                    input_ids = input_ids.to(self.device)
                    token_type_ids = token_type_ids.to(self.device)
                    attention_mask = attention_mask.to(self.device)
                    labels = labels.to(self.device)
                    outputs = self.model(country_mask=country_mask, input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
                    # print(labels.size())
                    loss = self.criterion(outputs, labels)

                    eval_loss += loss.item()
                    global_step += 1

                    eval_recall, eval_prec = updatemetics(eval_recall, eval_prec, outputs.data.cpu().numpy(), labels.data.cpu().numpy(), False)

                    del country_mask, input_ids, token_type_ids, attention_mask, labels
                    torch.cuda.empty_cache()


                for k, item in eval_recall.items():
                    if item[1] != 0:
                        print('eval recall class%d:'%k, item[0]/item[1], end=' ')
                print('\n')

                for k, item in eval_prec.items():
                    if item[1] != 0:
                        print('eval prec class%d:'%k, item[0]/item[1], end=' ')
                print('\n')

                for k, _ in prec.items():
                    if eval_recall[k][1] != 0 and eval_prec[k][1] != 0:
                        recall_ = eval_recall[k][0]/eval_recall[k][1]
                        prec_ = eval_prec[k][0]/eval_prec[k][1]
                        if recall_+prec_!= 0:
                            print('eval f1 class%d:'%k, 2*recall_*prec_/(recall_+prec_), end=' ')
                print('\n')
                eval_loss /= step
                print('eval LOSS:', eval_loss)
                self.testing_writter.add_scalar('loss', eval_loss, self.global_step)
                
                if best_eval_loss is None or eval_loss < best_eval_loss:
                    logging.info(f"Saving model with eval_loss = {eval_loss}...\n")
                    torch.save({
                        "recall": recall,
                        "prec": prec,
                        "eval_recall": eval_recall,
                        "eval_prec": eval_prec,
                        "epoch": epoch,
                        "step": step,
                        "global_step": global_step,
                        "optimizer": optimizer.state_dict(),
                        "model": self.model.state_dict(),
                        "train_loss": train_loss,
                        "eval_loss": best_eval_loss
                    }, os.path.join(output_dir, f"epoch{epoch}_global-step{global_step}"))
                    best_eval_loss = eval_loss
                logging.info(f"epoch = {epoch}, step = {step}\n")
                logging.info(f"train_loss = {train_loss}\n")
                logging.info(f"eval_loss = {eval_loss}\n")
                print(f"epoch{epoch}_global-step{global_step}", ' SAVED ')
                self.model.train()


            train_loss = 0
            recall = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
            prec = {0:[0, 0], 1:[0, 0], 2:[0, 0], 3:[0, 0], 4:[0, 0]}
            for step, example in enumerate(self.train_dataloader):
                # get outputs
                country_mask, input_ids, token_type_ids, attention_mask, labels = example
                country_mask = country_mask.to(self.device)
                input_ids = input_ids.to(self.device)
                token_type_ids = token_type_ids.to(self.device)
                attention_mask = attention_mask.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(country_mask=country_mask, input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
                # print(labels.size())
                loss = self.criterion(outputs, labels)

                # back prop & update variables
                # if grad_accumulation_steps > 1:
                #     loss = loss / grad_accumulation_steps
                loss.backward()
                # if (step + 1) % grad_accumulation_steps == 0:)

                optimizer.step()
                optimizer.zero_grad()
                train_loss += loss.item()
                global_step += 1

                recall, prec = updatemetics(recall, prec, outputs.data.cpu().numpy(), labels.data.cpu().numpy(), False)
 

                del country_mask, input_ids, token_type_ids, attention_mask, labels
                torch.cuda.empty_cache()
                

                if (step+1)%100 == 0:
                    print('TRAINIG LOSS:', step, train_loss/(step+1))
                    for k, item in recall.items():
                        if item[1] != 0:
                            print('TRAING recall class%d:'%k, item[0]/item[1], end=' ')
                    print('\n')

                    for k, item in prec.items():
                        if item[1] != 0:
                            print('TRAING prec class%d:'%k, item[0]/item[1], end=' ')
                    print('\n')

                    for k, _ in prec.items():
                        if recall[k][1] != 0 and prec[k][1] != 0:
                            recall_ = recall[k][0]/recall[k][1]
                            prec_ = prec[k][0]/prec[k][1]
                            if recall_+prec_!= 0:
                                print('TRAING f1 class%d:'%k, 2*recall_*prec_/(recall_+prec_), end=' ')
                    print('\n')

                self.training_writter.add_scalar('loss', loss.item(), self.global_step)

                self.global_step += 1
            

                # do eval
            # if epoch % eval_steps == 0 or step == len(self.dev_dataloader):


## metrics

In [ ]:
def updatemetics(recall, prec, outputs, labels, print=True):
    max_outputs = np.argmax(outputs, axis=-1)
    for data in zip(max_outputs, labels):
        out, l = data
        if out == l:
            recall[l][0] += 1
            recall[l][1] += 1
        else:
            recall[l][1] += 1

        if out == l:
            prec[out][0] += 1
            prec[out][1] += 1
        else:
            prec[out][1] += 1
    
    if print:
        for k, item in recall.items():
            if item[1] != 0:
                print('recall class%d:'%k, item[0]/item[1], end=' ')
        print('\n')

        for k, item in prec.items():
            if item[1] != 0:
                print('prec class%d:'%k, item[0]/item[1], end=' ')
        print('\n')

        for k, _ in prec.items():
            if recall[k][1] != 0 and prec[k][1] != 0:
                recall_ = recall[k][0]/recall[k][1]
                prec_ = prec[k][0]/prec[k][1]
                if recall_+prec_!= 0:
                    print('f1 class%d:'%k, 2*recall_*prec_/(recall_+prec_), end=' ')
        print('\n')
    
    return recall, prec

## main

In [ ]:
dataset_training = Country_purpose_dataset(training = True)
dataset_eval = Country_purpose_dataset(training = False)
batch_size = 4

training_data_loader = DataLoader(dataset=dataset_training, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)
eval_data_loader = DataLoader(dataset=dataset_eval, batch_size=batch_size, 
                               shuffle=False, collate_fn=collate_fn)

DATA AMOUNT: 2258
167
SKIP
SKIP
SKIP
SKIP NUMBER 170
SKIP LIST ['21435', '4108', '4452', '25269', '27425', '29490', '31836', '1167', '15614', '2558', '4289', '35257', '19871', '16333', '17670', '27969', '2258', '14885', '1253', '29480', '23742', '28342', '21413', '38283', '10118', '28366', '20145', '33432', '5133', '19372', '17471', '33883', '24771', '20039', '144', '26823', '14531', '33615', '13859', '22562', '21732', '17122', '12961', '17224', '25342', '16877', '6422', '33437', '24935', '31734', '21695', '19579', '19852', '1226', '15936', '38656', '319', '22942', '37233', '38479', '28615', '27836', '16243', '27204', '26418', '33616', '39152', '38771', '2505', '13658', '21152', '39431', '4151', '37171', '28628', '25225', '6879', '27790', '22558', '6263', '24363', '26791', '2866', '1380', '22940', '4161', '6627', '37193', '275', '7214', '7284', '38211', '16982', '2809', '28064', '31212', '1168', '13986', '27726', '624', '20444', '24642', '1228', '27203', '26486', '8842', '38080', '6981

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

True
Using device: cuda


In [ ]:
bert = HDCTModel.from_pretrained('bert-base-uncased').to(device)
# bert = HDCTModel(hsize = 600)


Some weights of HDCTModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.fc.weight', 'bert.fc.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert1 = HDCTModel.from_pretrained('bert-base-uncased').to(device)

Some weights of HDCTModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.fc.weight', 'bert.fc.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(bert, device, training_data_loader, eval_data_loader)

In [ ]:
        # country_hidden_total = torch.sum(last_hidden*country_mask.unsqueeze(-1), dim=1).squeeze()# (batch_size, hidden_size)
        # country_hidden_n = torch.sum(country_mask, dim=-1).view(last_hidden.size(0), 1)
        # country_hidden = country_hidden_total/country_hidden_n

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(lr)
trainer.train()


1e-06
0
eval recall class0: 0.047619047619047616 eval recall class1: 0.04838709677419355 eval recall class2: 0.5094339622641509 eval recall class3: 0.25 eval recall class4: 0.06666666666666667 

eval prec class0: 0.20408163265306123 eval prec class1: 0.3333333333333333 eval prec class2: 0.14210526315789473 eval prec class3: 0.07920792079207921 eval prec class4: 0.043478260869565216 

eval f1 class0: 0.07722007722007722 eval f1 class1: 0.08450704225352113 eval f1 class2: 0.2222222222222222 eval f1 class3: 0.12030075187969924 eval f1 class4: 0.052631578947368425 

eval LOSS: 1.726101217062577
epoch0_global-step93  SAVED 
TRAINIG LOSS: 99 1.4882776629924774
TRAING recall class0: 0.65625 TRAING recall class1: 0.05 TRAING recall class2: 0.11290322580645161 TRAING recall class3: 0.06818181818181818 TRAING recall class4: 0.0 

TRAING prec class0: 0.5610687022900763 TRAING prec class1: 0.2727272727272727 TRAING prec class2: 0.08536585365853659 TRAING prec class3: 0.07692307692307693 TRAING pre

KeyboardInterrupt: ignored

#inference on the entire dataset

## entire training

In [ ]:
epochs = 30
eval_steps = 1
output_dir = 'purpose_classification_new/entire_training' # output directory to save the best models
bert = HDCTModel.from_pretrained('bert-base-uncased').to(device)

dataset_training = Country_purpose_dataset(training = True, entire=True)
dataset_eval = Country_purpose_dataset(training = False)
batch_size = 4

training_data_loader = DataLoader(dataset=dataset_training, batch_size=batch_size, 
                               shuffle=True, collate_fn=collate_fn)
eval_data_loader = DataLoader(dataset=dataset_eval, batch_size=batch_size, 
                               shuffle=False, collate_fn=collate_fn)

trainer = Trainer(bert, device, training_data_loader, eval_data_loader)
trainer.train()

Some weights of HDCTModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.fc.weight', 'bert.fc.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DATA AMOUNT: 3737
259
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP NUMBER 267
SKIP LIST ['28615', '33615', '27203', '22092', '6320', '182', '18178', '1618', '29315', '31201', '21732', '22987', '36191', '19852', '21415', '3256', '200', '37233', '27230', '28714', '1172', '17225', '7065', '22995', '22391', '22335', '28336', '14531', '3264', '4452', '481', '26495', '28487', '1226', '38326', '275', '26188', '18435', '2505', '26418', '25949', '31750', '35028', '1359', '20317', '24805', '15936', '22942', '2601', '38980', '6981', '28628', '25342', '15834', '27205', '20038', '26929', '14756', '23121', '24935', '22613', '8842', '16877', '28864', '39103', '22625', '24470', '147', '320', '7214', '3006', '22622', '33437', '23742', '27791', '17122', '6263', '20444', '36268', '13369', '366', '22558', '16243', '17471', '27204', '34339', '30514', '24439', '7758', '19372', '2610', '38771', '953', '20039', '31537', '3255', '13986', '26486', '3327', '26897', '27726', '26963', '14885', '16717', '21152', '2

KeyboardInterrupt: ignored

## entire dataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class Inference_dataset(Dataset):
    def __init__(self):

        self.preprocess_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        with open('purpose_classification/common_country_sample_idx.json', 'r') as f:
            self.idxes = json.load(f)

        print('DATA AMOUNT:', len(self.idxes))
            
        # self.idx_target = list(self.data) # data_idx, target

        path2newfiles = 'purpose_classification'
        file_handle = open('/'.join([path2newfiles, 'global_country_idx2data.json']), 'r')
        self.idx2data = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'common_country_sample_idx.json']), 'r')
        self.common_country_sample_idx = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country_token.json']), 'r')
        self.token = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country_data.json']), 'r')
        self.pos = json.load(file_handle)
        file_handle.close()

        file_handle = open('/'.join([path2newfiles, 'global_country2idx.json']), 'r')
        self.global_country2idx = json.load(file_handle)
        file_handle.close()

        # get full list of country for each cat_name+row_idx
        done = ["reviews_Musical_Instruments_5",  "reviews_Automotive_5",
                "reviews_Cell_Phones_and_Accessories_5", "reviews_Toys_and_Games_5", 
                "reviews_Office_Products_5", "reviews_Pet_Supplies_5",
                "reviews_Digital_Music_5", 'reviews_Tools_and_Home_Improvement_5',
                "reviews_Clothing_Shoes_and_Jewelry_5", "reviews_Amazon_Instant_Video_5"]

        try:
            with open('purpose_classification_new/fail.json', 'w') as f:
                self.data = json.load(f)
        except:
            print('FAIL TO LOAD EXISTING DATA')
            find_complete_country_list = {}
            incomplete_data = []
            for index in self.idxes:
                # import pdb
                # pdb.set_trace()
                info = self.idx2data[str(index)]
                text = self.token[str(info['category'])][str(info['row_idx'])][1]
                incomplete_data.append((index, text, info))
                find_complete_country_list[(info['category'], int(info['row_idx']))] = 0
            
            for name in done:
                file_path = '/'.join([name+'_all', name])
                for row_idx, data in enumerate(open(file_path+'_feature.json', 'r')):
                    if (name, row_idx) in find_complete_country_list:
                        feature = json.loads(data)
                        assert len(feature['COUNTRY']) > 0
                        find_complete_country_list[(name, row_idx)] = feature['COUNTRY']
            
            # combine all data to self.data
            raw_data = []
            skip_list = []
            skip_c = 0
            for data in incomplete_data:
                index, text, info = data
                assert len(find_complete_country_list[(info['category'], int(info['row_idx']))]) != 0
                country_list = find_complete_country_list[(info['category'], info['row_idx'])]

                # if  self.pos[str(index)]['end'] >= 400 or int(info['country_idx']) >= 6 or index in ['22154', '26418', '29455', '7214', '500', '7284', '24013', '6981', '2850']:
                if self.pos[str(index)]['end'] >= 400 or index in ['22154', '26418', '29455', '7214', '500', '7284', '24013', '6981', '2850']:
                    skip_list.append(index)
                    skip_c += 1
                    continue
                raw_data.append((index, 0, text, info, country_list, self.pos[str(index)]['start'], self.pos[str(index)]['end'])) # data_idx, target, text, info(info['country_idx'], info['country_name']), country_list, start, end
            print(skip_c)

            self.data = []

            for r_d in raw_data:
                if self.check_if_tokenize_can_find_country(r_d, r_d[2]):
                    self.data.append(r_d)
                else:
                    skip_c += 1
                    skip_list.append(r_d[0])
            
            print('SKIP NUMBER', skip_c)
            print('SKIP LIST', skip_list)

            with open('purpose_classification_new/entire_training/all_dataset_complete_data.json', 'w') as f:
                json.dump(self.data, f)
    
            with open('purpose_classification_new/entire_training/all_dataset_skip_list.json', 'w') as f:
                json.dump(skip_list, f)

    def check_if_tokenize_can_find_country(self, r_d, text):
            d = {}
            d['country_name'] = r_d[3]['country_name']
            d['country_idx'] = r_d[3]['country_idx']
            d['country_list'] = r_d[-3]
            d['start'] = r_d[-2]
            d['end'] = r_d[-1]
            tokens = self.preprocess_tokenizer(text.lower(), return_tensors="pt", padding='max_length', max_length=512, truncation=True)
            input_ids, _, _ = tokens['input_ids'], tokens['token_type_ids'], tokens['attention_mask']
            start, end = find_country_idx(d, self.preprocess_tokenizer.convert_ids_to_tokens(input_ids[0]))

            if start == 1000 and end == 1000:
                return False
            else:
                return True

    def __getitem__(self, idx):
        # (text, target, country_name, country_idx, country_list)
        return {'index':self.data[idx][0], 'text':self.data[idx][2], 'target':int(self.data[idx][1])-1, 'country_name':self.data[idx][3]['country_name'], 
                'country_idx':self.data[idx][3]['country_idx'], 'country_list':self.data[idx][-3], 'start':self.data[idx][-2], 'end':self.data[idx][-1]}
    
    def __len__(self):
        return len(self.data)

def find_country_idx(d, tokenized_sent):
    country_name, country_idx, country_list, start, end = d['country_name'], d['country_idx'], d['country_list'], d['start'], d['end']
    words_length = len(country_name.split(' '))
    record = []
    new_sent = []
    # print(tokenized_sent)
    for idx, t in enumerate(tokenized_sent):
        if idx == 0:
            last_t = t
            record.append(1)
            new_sent.append(t)
            continue
        
        if t.startswith('##'):
            record[-1] += 1
            new_sent[-1] = new_sent[-1] + t[2:]
        else:
            record.append(1)
            new_sent.append(t)

    assert len(record) == len(new_sent)

    pointer = 0
    start = 0
    loop = 0
    while 1:
        cur_words = country_list[pointer].lower().split(' ')
        cur_words_len = len(cur_words)
        # print('WORD TO FIND:', cur_words)
        for i in range(start, len(new_sent)-cur_words_len, 1):

            if len(re.sub("[^A-Za-z']+", '', new_sent[i])) == 0:
                continue

            w2compare = [re.sub("[^A-Za-z']+", '', new_sent[i])]
        
            j = i+1
            while len(w2compare) < cur_words_len:
                if len(re.sub("[^A-Za-z']+", '', new_sent[j])) > 0:
                    w2compare.append(new_sent[j])
                j += 1
            
            if w2compare == cur_words:
                if country_idx != pointer:
                    # print('CURRENT WORD==WORD TO FIND', new_sent[i:j], cur_words, country_idx, pointer, i)
                    pointer += 1
                    start = i+cur_words_len
                    break
                else:
                    # print('CURRENT WORD==WORD TO FIND and RETURN', new_sent[i:j], cur_words, country_idx, pointer, i)
                    start_ = sum(record[:i])
                    end_ = sum(record[:j])
                    # print(tokenized_sent[start_:end_], new_sent[i:j])
                    return start_, end_

        if i == len(new_sent)-cur_words_len-1:
            print('SKIP')
            return 1000, 1000

    if start_ == 0 and end_ == 0:
        print('ERROR')
        import pdb
        pdb.set_trace()
    
    return start_, end_

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
def collate_fn(data):
    # tokenized sentence batchxsequence x3
    # target batchx1
    # country_mask batchxsequence
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    targets = []
    country_mask = torch.zeros((len(data), 512))
    indexes = []
    for idx,d in enumerate(data):
        # print(d['target'])
        tokens = tokenizer(d['text'].lower(), return_tensors="pt", padding='max_length', max_length=512, truncation=True)
        input_ids, token_type_ids, attention_mask = tokens['input_ids'], tokens['token_type_ids'], tokens['attention_mask']
        start, end = find_country_idx(d, tokenizer.convert_ids_to_tokens(input_ids[0]))
        if start == 1000 and end == 1000:
            print('ERROR')
            import pdb
            pdb.set_trace()
        country_mask[idx, start:end] = 1
        indexes.append(d['index'])

        input_ids_list.append(input_ids)
        token_type_ids_list.append(token_type_ids)
        attention_mask_list.append(attention_mask)

    input_ids_list = torch.cat(input_ids_list, dim=0)
    token_type_ids_list = torch.cat(token_type_ids_list, dim=0)
    attention_mask_list = torch.cat(attention_mask_list, dim=0)
    
    return country_mask, input_ids_list, token_type_ids_list, attention_mask_list, indexes
    


In [ ]:
all_dataset = Inference_dataset()

batch_size = 4

all_data_loader = DataLoader(dataset=all_dataset, batch_size=batch_size, 
                               shuffle=False, collate_fn=collate_fn)


DATA AMOUNT: 31278
FAIL TO LOAD EXISTING DATA
2207
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP
SKIP NUMBER 2318
SKIP LIST [144, 305, 537, 544, 986, 1032, 1186, 1187, 1226, 1247, 1248, 1249, 1505, 2258, 2366, 2376, 2610, 2661, 2738, 2741, 2783, 2784, 2816, 2817, 2837, 2866, 2895, 2987, 3042, 3145, 3146, 3179, 3380, 3752, 3865, 3922, 4131, 4146, 4153, 4158, 4205, 4233, 4278, 4289, 4290, 4301, 4537, 4593, 4605, 4639, 4642, 4909, 4925, 5133, 5141, 5330, 5781, 5884, 6328, 6421, 6688, 6819,

## entire inference

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

True
Using device: cuda


In [ ]:
bert = HDCTModel.from_pretrained('bert-base-uncased').to(device)
metadata = torch.load('purpose_classification_new/entire_training/epoch12_global-step11612')
bert.load_state_dict(metadata['model'])

Some weights of HDCTModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.fc.weight', 'bert.fc.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
def inference_entire_dataset(model, all_dataloader):
    model.eval()
    data_results = {}
    stats = {0:0, 1:0, 2:0, 3:0, 4:0}
    for step, example in enumerate(all_dataloader):
        # get outputs
        country_mask, input_ids, token_type_ids, attention_mask, indexes = example
        country_mask = country_mask.to(device)
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        # labels = labels.to(device)
        outputs = model(country_mask=country_mask, input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

        # print(outputs.size())
        prediction = torch.argmax(outputs, dim=-1).squeeze().cpu().data.numpy()

        for index, pred in zip(indexes, prediction):
            if pred != 0:
                print(pred)
            data_results[index] = int(pred)
            stats[int(pred)] += 1

        del country_mask, input_ids, token_type_ids, attention_mask, indexes
        torch.cuda.empty_cache()

        if (step+1)%1000 == 0:
            print(step)
            with open('purpose_classification_new/entire_training/entire_dataset_inference_results.json', 'w') as f:
                json.dump(data_results, f)
    
    with open('purpose_classification_new/entire_training/entire_dataset_inference_results.json', 'w') as f:
        json.dump(data_results, f)

    print(stats)
    print(data_results)


## Stats analysis

In [ ]:
country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 
                      'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'],  'australia':['australia'], 
                      'italy':['italy'], 'france':['france'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'brazil':['brazil'], 'korea':['korea'], 
                      'afghanistan':['afghanistan'], 'spain':['spain'], 'new zealand':['new zealand'], 
                      'singapore':['singapore'], 'iraq':['iraq'], 'russia':['russia'], 'switzerland':['switzerland']}


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

path2newfiles = 'purpose_classification'
file_handle = open('/'.join([path2newfiles, 'global_country_idx2data.json']), 'r')
idx2data = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'common_country_sample_idx.json']), 'r')
common_country_sample_idx = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_token.json']), 'r')
token = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country_data.json']), 'r')
pos = json.load(file_handle)
file_handle.close()

file_handle = open('/'.join([path2newfiles, 'global_country2idx.json']), 'r')
global_country2idx = json.load(file_handle)
file_handle.close()

In [ ]:

with open('purpose_classification_new/entire_training/entire_dataset_inference_results.json', 'r') as f:
    ruiqi = json.load(f)

# idx2country:
idx2country = {}
for k, item in global_country2idx.items():
    for i in item:
        idx2country[int(i)] = k
    
total = copy.deepcopy(ruiqi)

print(len(total))

dict_ = {}
for k, item in total.items():
    print(k, item)
    if idx2country[int(k)] not in word2label_country:
        print(idx2country[int(k)])
        continue
    country = word2label_country[idx2country[int(k)]]
    if country not in dict_:
        dict_[country] = {item:1}
    else:
        if item not in dict_[country]:
            dict_[country][item] = 1
        else:
            dict_[country][item] += 1

print(dict_)

Streaming output truncated to the last 5000 lines.
370 3
674 2
735 2
1333 2
1832 3
2334 2
2539 0
2628 2
2629 2
2651 3
2988 2
3553 1
3760 1
3953 3
4230 2
4248 2
4264 3
4300 3
4469 3
4993 3
4994 3
5001 2
5047 1
5180 2
5334 2
5335 2
5336 3
5337 3
5653 0
5774 2
5776 3
6485 0
6611 0
6806 0
6807 0
6859 2
6947 2
7004 0
7005 0
7225 0
7257 2
8237 0
8238 0
9017 0
9018 0
10000 2
10127 0
11754 3
11761 3
14686 0
15105 0
15331 0
15600 3
15844 2
15846 0
17625 2
18799 0
18943 2
19956 0
20349 3
20351 1
20720 0
21160 0
21195 3
21208 2
21489 0
21638 0
21693 0
21694 0
21696 0
21697 0
21702 0
21978 0
22239 0
22259 2
22693 2
23187 2
23513 3
23678 0
23679 0
23692 2
23880 3
24181 3
24199 2
24204 2
24550 3
24552 3
24671 2
24925 3
25120 2
25151 3
25293 2
25550 2
25942 3
25974 2
26256 3
26407 3
26954 0
27379 3
27385 3
27386 3
27387 3
27389 3
27390 3
27391 0
27393 3
27395 3
27397 3
27398 3
27400 3
27401 3
27402 3
27403 3
27404 3
27407 3
27493 3
27572 3
27728 3
27729 3
27730 3
27731 3
27897 3
27923 1
27926 3
28002

In [ ]:
stat = {0:0, 1:0, 2:0, 3:0, 4:0}
for k, item in ruiqi.items():
    stat[int(item)] += 1

print(stat)


{0: 14751, 1: 3797, 2: 4981, 3: 4762, 4: 669}
